In [84]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import numpy as np

## Import Price Data from Yahoo Finance

In [85]:
stocks = ['TSLA', 'GOOG', 'BA', 'DPZ']

START_DATE = dt.datetime(1970,1,1)
END_DATE = dt.datetime(2020,7,31)

df = pd.DataFrame()

for stock in stocks:
    df[stock] = web.DataReader(stock, 'yahoo',START_DATE,END_DATE)['Adj Close']

df

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-29,23.889999,226.281845,49.364201,9.381756
2010-06-30,23.830000,221.644226,49.137108,9.646392
2010-07-01,21.959999,218.924423,48.753410,9.654928
2010-07-02,19.200001,217.459900,48.502819,9.817122
2010-07-06,16.110001,217.220810,48.048664,9.526877
...,...,...,...,...
2020-07-27,1539.599976,1530.199951,170.210007,389.609985
2020-07-28,1476.489990,1500.339966,170.839996,385.420013
2020-07-29,1499.109985,1522.020020,166.009995,385.149994


## Compute Daily Returns

In [86]:
daily_returns = df.pct_change()
daily_returns

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-29,NaN,NaN,NaN,NaN
2010-06-30,-0.002511,-0.020495,-0.004600,0.028208
2010-07-01,-0.078473,-0.012271,-0.007809,0.000885
2010-07-02,-0.125683,-0.006690,-0.005140,0.016799
2010-07-06,-0.160938,-0.001099,-0.009363,-0.029565
...,...,...,...,...
2020-07-27,0.086521,0.012124,-0.020430,0.006978
2020-07-28,-0.040991,-0.019514,0.003701,-0.010754
2020-07-29,0.015320,0.014450,-0.028272,-0.000701


## Compute Cumulative Returns (Normalized Prices)

In [87]:
# method 1: cumulative returns calculated with cumprod
cumulative_returns = (1 + daily_returns).cumprod()
cumulative_returns.fillna(1, inplace=True)
cumulative_returns

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-29,1.000000,1.000000,1.000000,1.000000
2010-06-30,0.997489,0.979505,0.995400,1.028208
2010-07-01,0.919213,0.967486,0.987627,1.029117
2010-07-02,0.803684,0.961013,0.982550,1.046406
2010-07-06,0.674341,0.959957,0.973350,1.015468
...,...,...,...,...
2020-07-27,64.445375,6.762363,3.448045,41.528472
2020-07-28,61.803685,6.630404,3.460808,41.081864
2020-07-29,62.750524,6.726214,3.362963,41.053082


In [88]:
# method 2: cumulative returns calculated with price normalization
df_copy = df.copy()
df_copy = df / df.iloc[0]
df_copy

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-29,1.000000,1.000000,1.000000,1.000000
2010-06-30,0.997489,0.979505,0.995400,1.028208
2010-07-01,0.919213,0.967486,0.987627,1.029117
2010-07-02,0.803684,0.961013,0.982550,1.046406
2010-07-06,0.674341,0.959957,0.973350,1.015468
...,...,...,...,...
2020-07-27,64.445375,6.762363,3.448045,41.528472
2020-07-28,61.803685,6.630404,3.460808,41.081864
2020-07-29,62.750524,6.726214,3.362963,41.053082


## Annualized Returns from Cumulative Returns of Daily Returns

$$\LARGE r_{annualized} = \{\Pi(1+r_{i})\}^{T/t} - 1$$

$\Pi(1+r_{i})$ = cumulative return <br>
$T$ = factor for converting the observations to annual (e.g., 12 for converting monthly to annual) <br>
$t$ = number of observations (either on a daily, monthly or even quarterly basis)

In [89]:
firstday = daily_returns.index[0] # the first row
lastday = daily_returns.index[-1] # the last row
diff = lastday - firstday
print("Date difference between first and last including weekends = {}(t1) days for 365(T1) days".format(diff.days))
print("Total days from the dataset excluding weekends = {}(t2) days for 252(T2) days".format(len(cumulative_returns.index)))

Date difference between first and last including weekends = 3685(t1) days for 365(T1) days
Total days from the dataset excluding weekends = 2541(t2) days for 252(T2) days


In [90]:
T1 = 365
t1 = 3685

annualized_returns1 = cumulative**(T1/t1) - 1
annualized_returns1

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-30,-0.000249,-0.002049,-0.000457,0.002759
2010-07-01,-0.008309,-0.003269,-0.001232,0.002847
2010-07-02,-0.021415,-0.003931,-0.001742,0.004503
2010-07-06,-0.038276,-0.004040,-0.002672,0.001522
2010-07-07,-0.040125,-0.000889,0.000408,0.003373
...,...,...,...,...
2020-07-27,0.510779,0.208430,0.130438,0.446430
2020-07-28,0.504528,0.206073,0.130852,0.444882
2020-07-29,0.506796,0.207788,0.127644,0.444782


In [91]:
T2 = 252
t2 = 2541

annualized_returns2 = cumulative**(T2/t2) - 1
annualized_returns2

,TSLA,GOOG,BA,DPZ
Date,,,,
2010-06-30,-0.000249,-0.002052,-0.000457,0.002763
2010-07-01,-0.008319,-0.003273,-0.001234,0.002850
2010-07-02,-0.021441,-0.003936,-0.001744,0.004509
2010-07-06,-0.038323,-0.004045,-0.002675,0.001523
2010-07-07,-0.040174,-0.000890,0.000408,0.003377
...,...,...,...,...
2020-07-27,0.511555,0.208715,0.130611,0.447095
2020-07-28,0.505294,0.206355,0.131025,0.445544
2020-07-29,0.507565,0.208072,0.127813,0.445444


## Compounded Annual Growth Rate (CAGR)

$\LARGE \text{CAGR =} \left(\frac{\text{Ending Balance}}{\text{Beginning Balance}}\right)^{1/\text{N}} - 1$

## Annualization of Daily Volatility (Standard Deviation)

$\LARGE \sigma_{\text{T}} = \sqrt{\text{T}} \times \sigma_{t} $

In [97]:
# daily standard deviation
daily_vol = daily_returns.std()
print("Daily Volatility")
print("="*25)
print(daily_vol)
print()

# annualization of daily standard deviation
T = 252
annual_vol = daily_vol * np.sqrt(T)
print("Annualized Volatility")
print("="*25)
print()
print(annual_vol)

Daily Volatility
TSLA    0.034762
GOOG    0.016272
BA      0.021907
DPZ     0.018397
dtype: float64

Annualized Volatility

TSLA    0.551837
GOOG    0.258307
BA      0.347759
DPZ     0.292039
dtype: float64


## Sharpe Ratio

$\LARGE \text{Sharpe Ratio = } \frac{R_{p} - R_{f}}{\sigma_{p}} $

In [110]:
summary = pd.DataFrame(index = stocks)
summary['CAGR'] = np.transpose(annualized_returns2.iloc[-1,:]).round(2)
summary['Annualized Vol'] = annual_vol.round(2)

# risk free rate is assumed to be 0%
rf = 0

# Sharpe ratio calculation
summary['Sharpe Ratio'] = ((summary['CAGR'] - rf)/summary['Annualized Vol']).round(2)
summary

,CAGR,Annualized Vol,Sharpe Ratio
TSLA,0.50,0.55,0.91
GOOG,0.20,0.26,0.77
BA,0.12,0.35,0.34
DPZ,0.45,0.29,1.55
